In [ ]:
import DatasetProcessor as dp
import mne
import glob
import os
import pandas as pd

In [ ]:
# config
DATA_IN = "../data/tuh_seizure"
DATA_OUT = "../data/processed"
DESIRED_CHANNELS = ["FP1", "FP2", "F3", "F4", "C3", "C4",
                    "P3", "P4", "O1", "O2", "F7", "F8", 
                    "T7", "P7", "T8", "P8", "FZ", "CZ", "PZ"]
TARGET_SFREQ=100
N_JOBS=2
SPLIT_LENGTH=10     # in Seconds
HIGHPASS=1
LOWPASS=50

In [ ]:
# get all edfs
edf_list = glob.glob(os.path.join(DATA_IN, "*", "*", "*", "*", "*", "*", "*.edf"))
df = pd.DataFrame({"EEG_Files": edf_list})
df

In [ ]:
one_edf = df.EEG_Files[115]
raw = mne.io.read_raw_edf(one_edf)
#raw.plot(title="Raw EEG", scalings=60e-6, duration=5)

In [ ]:
et = dp.EEGTransformer(desired_channels=DESIRED_CHANNELS, target_sfreq=TARGET_SFREQ, 
                       lowpass=LOWPASS, highpass=HIGHPASS, n_jobs=N_JOBS)
proc_raw, ch_mapping = et.transformEEG(one_edf)
#ch_mapping

In [ ]:
#proc_raw.plot(title="Processed EEG", scalings=60e-6, duration=5)
target = os.path.split(one_edf)[-1].split("_")[0]

target_stem = os.path.join(DATA_OUT, target)

idx = len(glob.glob(f"{target_stem}_split_*.edf")) + 1

filename = target + "_split_" + str(idx) + ".edf"

print(f"Target stem: {target_stem}")
print(f"Target: {target}")
print(f"Idx = {idx}")
print(f"Filename = {filename}") 

In [ ]:
sas = dp.SplitAndSaveEEG(split_length=SPLIT_LENGTH, data_out=DATA_OUT)
sas.split_EEG(proc_raw, target=target)

In [ ]:
import glob
glob.glob(DATA_OUT + "/*")
